<a href="https://colab.research.google.com/github/gibranfp/CursoAprendizajeProfundo/blob/master/notebooks/1a_perceptron_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neuronas artificiales 
---
La neurona artificial es un modelo simplificado de la neurona natural, la cual trata de imitar 3 aspectos principales: 

1. La fuerza sináptica que pondera los impulsos recibidos
2. La acumulación de estos impulsos ponderados 
3. La activación de la neurona que produce un impulso de respuesta a su salida. 

La primera neurona artificial fue la llamada Unidad de Umbral Lineal, propuesta en 1943 por Warren McCulloch y Walter Pitts, la cual presupone que tanto los valores de los atributos de entrada como los valores de salida son binarios.

In [1]:
import torch
import matplotlib.pyplot as plt

## Unidad de umbral lineal
La operación que lleva a cabo una neurona artificial está dada por la suma pesada evaluada en una función de activación $\phi$.  Una de las primeras funciones de activación utilizadas fue la escalón unitario, definida como

$
\phi(x) = \begin{cases} 1, & \text{si } x > 0\\0, & \text{en caso contrario}\end{cases}
$

Esta se puede llevar a cabo con la siguiente función de Python:

In [2]:
def escalon(z):
    return (z > 0.).float()

Por su parte, la suma pesada simplemente consiste en multiplicar cada entrada por su correspondiente peso y sumarle el sesgo. Esto lo podemos expresar como

$
z = w_1 \cdot x_1 + w_2 \cdot x_2 + \cdots + w_d \cdot x_d + b 
$

En su forma vectorial

$
z = \mathbf{w}^T \mathbf{x} + b
$

Para realizar esto con PyTorch podemos usar `@` para realizar la multiplicación de los vectores `w` y `x` y sumar `b` de la siguiente manera `z = w @ x + b`. Así, la operación de la neurona completa sería:

In [3]:
def neurona(x, w, b):
    z = w @ x + b
    a = escalon(z)

    return a

### OR ($\lor$)
Esta neurona es capaz de aproximar el operador OR, cuya salida es 1 cuando al menos 1 de las 2 entradas es 1:


| $x_1$ | $x_2$ | $y$
| ------------- |:-------------:| -----:|
|0 |0 |0|
|0 |1 |1|
|1 |0 |1|
|1 |1 |1|

La neurona recibe 2 valores binarios como entrada y produce un valor binario como salida. Específicamente, la neurona calcularía

$
\hat{y} = \phi(w_1 \cdot x_1 + w_2 \cdot x_2 + b)
$

Para poder aproximar la operación OR es necesario encontrar los valores apropiados de $w_1$, $w_2$ y $b$. Una posible elección sería 10, 10 y -5 respectivamente. Verifiquemos estos valores:

In [4]:
X = torch.tensor([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])
w = torch.tensor([10., 10.])
b = -5.

print('-----------------------------')
print('x_1 \tx_2 \ty_hat')
print('-----------------------------')
for i in range(X.shape[0]):
    y_hat = neurona(X[i, :], w, b)
    print('{0} \t{1}\t{2}'.format(X[i, 0], X[i, 1], y_hat))

-----------------------------
x_1 	x_2 	y_hat
-----------------------------
0.0 	0.0	0.0
0.0 	1.0	1.0
1.0 	0.0	1.0
1.0 	1.0	1.0


### AND ($\land$)
De forma similar, podemos aproximar la operación AND:


| $x_1$ | $x_2$ | $y$
| ------------- |:-------------:| -----:|
|0 |0 |0|
|0 |1 |0|
|1 |0 |0|
|1 |1 |1|

Nuevamente, debemos encontrar los valores apropiados para los pesos y el sesgo. Probemos con $w_1 = 10$, $w_2 = 10$ y $b = -15$.

In [5]:
w = torch.tensor([10., 10.])
b = -15.

print('-----------------------------')
print('x_1 \tx_2 \ty_hat')
print('-----------------------------')
for i in range(X.shape[0]):
    y_hat = neurona(X[i, :], w, b)
    print('{0} \t{1}\t{2}'.format(X[i, 0], X[i, 1], y_hat))

-----------------------------
x_1 	x_2 	y_hat
-----------------------------
0.0 	0.0	0.0
0.0 	1.0	0.0
1.0 	0.0	0.0
1.0 	1.0	1.0


### NAND 
También podemos aproximar la negación de la operación AND (NAND):


| $x_1$ | $x_2$ | $y$
| ------------- |:-------------:| -----:|
|0 |0 |1|
|0 |1 |1|
|1 |0 |1|
|1 |1 |0|

En este caso, los valores para los pesos y el sesgo son $w_1 = -10$, $w_2 = -10$ y $b = 15$.

In [6]:
w = torch.tensor([-10., -10.])
b = 15.

print('-----------------------------')
print('x_1 \tx_2 \ty_hat')
print('-----------------------------')
for i in range(X.shape[0]):
    y_hat = neurona(X[i, :], w, b)
    print('{0} \t{1}\t{2}'.format(X[i, 0], X[i, 1], y_hat))

-----------------------------
x_1 	x_2 	y_hat
-----------------------------
0.0 	0.0	1.0
0.0 	1.0	1.0
1.0 	0.0	1.0
1.0 	1.0	0.0


## Algoritmo del perceptrón


In [7]:
def perceptron(X, y, n_epochs = 10):
    w_new = torch.zeros(X.shape[1])
    b_new = 0
    for i in range(n_epochs):
        serr = 0.0
        for j in range(X.shape[0]):
            w_old = w_new
            b_old = b_new
            
            y_hat = neurona(X[j], w_old, b_old)
            error = y[j] - y_hat
           
            w_new = w_old + error * X[j]     
            b_new = b_old + error
           
            serr += torch.abs(error)
        print("Epoch {0}: error = {1}".format(i, serr / float(X.shape[0])))

    return w_new, b_new

### Aprendiendo la operación OR

Probemos el algoritmo del perceptrón para aprender la operación lógica OR.

In [8]:
X = torch.tensor([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])
y_or = torch.tensor([0., 1., 1., 1.]) 

w, b = perceptron(X, y_or)

print('\nw_1 = {0}, w_2 = {1}, b = {2}'.format(w[0], w[1], b))
print('-----------------------------')
print('x_1 \tx_2 \t y\ty_hat')
print('-----------------------------')
for i in range(X.shape[0]):
    y_hat = neurona(X[i], w, b)
    print('{0}\t{1}\t{2}\t{3}'.format(X[i, 0], X[i, 1], y_or[i], y_hat))

Epoch 0: error = 0.25
Epoch 1: error = 0.5
Epoch 2: error = 0.25
Epoch 3: error = 0.0
Epoch 4: error = 0.0
Epoch 5: error = 0.0
Epoch 6: error = 0.0
Epoch 7: error = 0.0
Epoch 8: error = 0.0
Epoch 9: error = 0.0

w_1 = 1.0, w_2 = 1.0, b = 0.0
-----------------------------
x_1 	x_2 	 y	y_hat
-----------------------------
0.0	0.0	0.0	0.0
0.0	1.0	1.0	1.0
1.0	0.0	1.0	1.0
1.0	1.0	1.0	1.0


### Aprendiendo la operación AND
Ahora veamos qué ocurre si en lugar de la operación OR tratamos de aprender la operación AND

In [9]:
y_and = torch.tensor([0., 0., 0., 1.])
w, b = perceptron(X, y_and)

print('\nw_1 = {0}, w_2 = {1}, b = {2}'.format(w[0], w[1], b))
print('-----------------------------')
print('x_1 \tx_2 \t y\ty_hat')
print('-----------------------------')
for i in range(X.shape[0]):
    y_hat = neurona(X[i], w, b)
    print('{0}\t{1}\t{2}\t{3}'.format(X[i, 0], X[i, 1], y_and[i], y_hat))

Epoch 0: error = 0.25
Epoch 1: error = 0.75
Epoch 2: error = 0.75
Epoch 3: error = 0.5
Epoch 4: error = 0.25
Epoch 5: error = 0.0
Epoch 6: error = 0.0
Epoch 7: error = 0.0
Epoch 8: error = 0.0
Epoch 9: error = 0.0

w_1 = 2.0, w_2 = 1.0, b = -2.0
-----------------------------
x_1 	x_2 	 y	y_hat
-----------------------------
0.0	0.0	0.0	0.0
0.0	1.0	0.0	0.0
1.0	0.0	0.0	0.0
1.0	1.0	1.0	1.0


## Aproximando funciones no lineales: XOR ($\oplus$)
Minsky y Papert mostraron que una neurona del tipo LTU no puede aproximar de forma precisa una función no lineal como la compuerta XOR ($\oplus$):


| $x_1$ | $x_2$ | $y$
| ------------- |:-------------:| -----:|
|0 |0 |0|
|0 |1 |1|
|1 |0 |1|
|1 |1 |0|

Sin embargo, es posible aproximar este tipo  combinando múltiples LTU conectadas en red. Por ejemplo, es posible llevar a cabo la operación XOR con operaciones OR, AND y NAND:

$
	  x_1 \mathbin{\oplus} x_2 = (x_1 \lor x_2) \land \neg(x_1 \land x_2)
	$  

Esto lo llevamos a cabo con la siguiente función:

In [10]:
def multicapa(x, W1, b1, W2, b2):
    a = escalon(W1.t() @ x + b1)
    return escalon(W2.t() @ a + b2)

Encontrando los valores de pesos y sesgos adecuados, podemos usar esta función para aproximar la operación XOR. Ya hemos encontrado los pesos y sesgos para las operaciones OR, AND y NAND, por lo que podemos usar estas neuronas con sus correspondientes pesos y sesgos. La red tendría 2 neuronas conectadas a las entradas que realizan las operaciones OR ($w_{11}^{\{1\}} = 10$, $w_{12}^{\{1\}} = 10$ y $b_1^{\{1\}} = -5$)  y NAND ($w_{21}^{\{1\}} = -10$, $w_{22}^{\{1\}} = -10$ y $b_2^{\{1\}} = 15$) respectivamente. La salida de estas 2 neuronas estarían conectadas a una tercera neurona que realiza la operacioón AND ($w_{11}^{\{2\}} = 10$, $w_{12}^{\{2\}} = 10$ y $b_1^{\{2\}} = -15$). En su forma matricial:

$$
\mathbf{W}^{\{1\}} = \left[\begin{matrix} 
10 & -10\\
10 & -10
\end{matrix}\right] 
$$

$$
\mathbf{b}^{\{1\}} = \left[\begin{matrix} 
-5 \\
15
\end{matrix}\right] 
$$

$$
\mathbf{W}^{\{2\}} = \left[\begin{matrix} 
10\\
10
\end{matrix}\right] 
$$

$$
\mathbf{b}^{\{2\}} = \left[\begin{matrix} 
-15\\
\end{matrix}\right] 
$$

In [11]:
y_xor = torch.tensor([0., 1., 1., 0.])
W1 = torch.tensor([[10., -10.], [10., -10.]])
b1 = torch.tensor([-5., 15.])

W2 = torch.tensor([[10.], [10.]])
b2 = torch.tensor([-15.])

print('W_1 = [{0}{1}], b_1 = {2}'.format(W1[0, :].tolist(), W1[1, :].tolist(), b1.tolist()))
print('W_2 = [{0}{1}], b_2 = {2}'.format(W2[0].tolist(), W2[1].tolist(), b2.tolist()))
print('-----------------------------')
print('x_1 \tx_2 \ty\ty_hat')
print('-----------------------------')
for i in range(X.shape[0]):
    y_hat = multicapa(X[i], W1, b1, W2, b2)
    print('{0}\t{1}\t{2}\t{3}'.format(X[i, 0], X[i, 1], y_xor[i], y_hat[0]))

W_1 = [[10.0, -10.0][10.0, -10.0]], b_1 = [-5.0, 15.0]
W_2 = [[10.0][10.0]], b_2 = [-15.0]
-----------------------------
x_1 	x_2 	y	y_hat
-----------------------------
0.0	0.0	0.0	0.0
0.0	1.0	1.0	1.0
1.0	0.0	1.0	1.0
1.0	1.0	0.0	0.0


## Ejercicio 
Desarrolla y evalúa una red de neuronas con funciones de activación escalón unitario que aproxime la operación XNOR ($\odot$) dada por

| $x_1$ | $x_2$ | $y$
| ------------- |:-------------:| -----:|
|0 |0 |1|
|0 |1 |0|
|1 |0 |0|
|1 |1 |1|
